# **REPORTE DE AUTODETRACCIONES FALABELLA.COM**

**Pasos**
1. Descargar archivo en formato .xls y luego abrirlo y convertirlo a .xlsx.
2. Todos los archivos deben estar en formato .xlsx.
3. Organizar todos los archivos en una sola carpeta.

### **Usuario**

In [12]:
# IMPORTANTE: 
# Solo se debe modificar el usuario, ruta (en caso sea diferente) y fechas de reporte. 
# Considerar que se debe sincronizar la carpeta 'MARKETPLACE PERÚ' antes de realizar la ejecución del script.

usuario = r'gfloress'
ruta = r'MARKETPLACE PERÚ - Documents'
carpeta = r'Detracción 2feb24'

### **Variables**

In [13]:
n_lote = '170000'

empresa = 'LINIO PERU SAC'
ruc = '20547836473'
nro_cuenta = '46080408'

#### **Librerías**

In [14]:
import os
import glob
import warnings

import numpy as np
import pandas as pd

warnings.filterwarnings(action='ignore')

pd.set_option('display.expand_frame_repr', False)
pd.options.display.float_format = '{:.2f}'.format

#### **Rutas**

In [15]:
# Rutas
ruta_base = r'C:\\Users\\' + usuario + r'\\Falabella\\' + ruta
ruta_postventa_fcom = ruta_base + r'\\1. Postventa\\3. F.com'
ruta_autodetracciones = ruta_postventa_fcom + r'\\1. Pagos\\1. Autodetracciones\\' + carpeta

#### **Archivos**

In [16]:
ventas_files = glob.glob(os.path.join(ruta_autodetracciones, "PEReporteVentas*.xlsx"))[0]
bob_files = glob.glob(os.path.join(ruta_autodetracciones, "BOB*.xlsx"))[0]
#autodetracciones_files = glob.glob(os.path.join(ruta_autodetracciones, "Auto*.xlsx"))[0]

### **Cuadro de conceptos**

In [17]:
conceptos = {
    'concepto': ['Shipping Fee', 'Free Ship', 'Commission', 'Mktg', 'Cancellation', 'pe_promo_item_pric', 'Consignment', 'Logistics rework'],
    'porcentaje': [0.10, 0.12, 0.10, 0.12, 0.10, 0.10, 0.12, 0.12],
    'codigo': ['024', '037', '024', '037', '024', '024', '037', '037']
}

conceptos = pd.DataFrame(conceptos)

### **Data**

In [18]:
reporte_ventas = pd.read_excel(ventas_files, sheet_name='PEReporteVentas', header=6)
bob_ventas = pd.read_excel(bob_files, sheet_name='Hoja1')

### **Proceso**

In [19]:
# Mover columna Foliador

(
    reporte_ventas
    .insert(
        reporte_ventas.columns.get_loc('Número de documento') + 1, 
        'Foliador', 
        reporte_ventas.pop('Foliador')
    )
)

In [20]:
# Transformación de data de reporte de ventas

a = (
    reporte_ventas
    .assign(**{
        'Foliador': lambda x: x['Foliador'].astype(str),
        'PE Tipo Comp Pago': lambda x: x['PE Tipo Comp Pago'].astype(str),
        'PE No Serie Comp Pago': lambda x: x['PE No Serie Comp Pago'].astype(str),
        'PE No Comp Pago': lambda x: x['PE No Comp Pago'].astype(str)
    })
    .sort_values(['Foliador'])  # Ordenar columna foliador
    .query("~Foliador.str.contains('^0120')")  # Quitar lo no timbrado
    .query("Foliador.notnull()")  # Quitar lo nulo
    #.query("Foliador!='nan'")
    .assign(**{
        'PE Tipo Comp Pago': '01',
        'PE No Serie Comp Pago': lambda x: x['Foliador'].str.split('-', n=1).str[0],
        'PE No Comp Pago': lambda x: x['Foliador'].str.split('-', n=1).str[1]
    })
    .query("`PE No Serie Comp Pago`=='F001'")  # Solo tener lo que empieza con F001
    .assign(**{
        'PE No Comp Pago': lambda x: x['PE No Comp Pago'].str.replace('(\.0$)', '').astype(int)
    })
    .sort_values(['PE No Comp Pago'])  # Ordenar
    .assign(**{
        'diff': lambda x: x['PE No Comp Pago'].diff().fillna(1).astype(int)
    })
)

In [21]:
# Transformación de data de reporte de ventas BOB

b = (
    bob_ventas
    .loc[:, [
        'invoice_id', 'invoice_issue_date', 'reciver_identification', 'reciver_name', 
        'total_paid', 'total_invoice_amount', 'currency_code_source', 'description', 'status'
    ]]
    .rename(
        columns={
            'invoice_id': 'Foliador', 'invoice_issue_date': 'Fecha',
            'reciver_identification': 'Número de identificación nacional', 'reciver_name': 'Nombre de la empresa',
            'total_paid': 'Importe (moneda extranjera)', 'total_invoice_amount': 'Custom Tax Total',
            'currency_code_source': 'Moneda: Símbolo de moneda'
        }
    )
    # Convertir a numero
    .assign(**{
        'Total Transaccion': lambda x: x['Importe (moneda extranjera)'] + x['Custom Tax Total']
    })
    # Agregar concepto
    .assign(description_2 = lambda x: x['description'].str.lower())
    .assign(concepto =
                lambda x:
                    np.select([
                        (x['description_2'].str.contains('shipping fee')==True),
                        (x['description_2'].str.contains('free ship')==True),
                        (x['description_2'].str.contains('commission')==True),
                        (x['description_2'].str.contains('mktg')==True),
                        (x['description_2'].str.contains('cancellation')==True),
                        (x['description_2'].str.contains('pe_promo_item_price')==True),
                        (x['description_2'].str.contains('consignment')==True),
                        (x['description_2'].str.contains('logistics rework')==True)
                    ],
                    [
                        'Shipping Fee',
                        'Free Ship',
                        'Commission',
                        'Mktg',
                        'Cancellation',
                        'pe_promo_item_pric',
                        'Consignment',
                        'Logistics rework'
                    ],
                    default='NA'
            )
    )
    .drop(['description_2'], axis=1)
    .merge(conceptos, on=['concepto'], how='left')
)

In [23]:
# Agregar números faltantes de documentos

missing_numbers = np.concatenate([
    np.arange(start, end + 1) if diff > 1 else np.array([start])
    for start, end, diff in zip(
        a['PE No Comp Pago'], 
        a['PE No Comp Pago'].shift(-1), 
        (a['PE No Comp Pago'].shift(-1) - a['PE No Comp Pago']).fillna(0)
        )
])

missing_numbers = np.sort(np.unique(missing_numbers))

existing_numbers = a['PE No Comp Pago'].to_list()

missing_numbers = np.setdiff1d(missing_numbers, existing_numbers)

In [24]:
# Formatear nuevas filas por agregar

a_ = (
    pd.DataFrame(missing_numbers, columns=['PE No Comp Pago'])
    .assign(**{
        'PE No Comp Pago': lambda x: x['PE No Comp Pago'].astype(str).str.replace('\.0$', '', regex=True).astype(int)
    })
    .assign(**{
        'Período contable: Fecha de inicio': a['Período contable: Fecha de inicio'].unique()[0],
        'Número de documento': lambda x: 'F001-' + x['PE No Comp Pago'].astype(str),
        'Foliador': lambda x: 'F001-' + x['PE No Comp Pago'].astype(str),
        'PE Tipo Comp Pago': '01',
        'PE No Serie Comp Pago': 'F001',
        'PE - Tipo Doc Identidad Cliente': 6,
        'Tipo de cambio': 1,
        'Fecha de vencimiento': pd.NaT
    })
    .merge(b.drop(['description','status'], axis=1).drop_duplicates(['Foliador'],keep='first'), on=['Foliador'], how='left')
    .loc[:, [
        'Período contable: Fecha de inicio', 'Número de documento', 'Foliador',
        'Fecha', 'Fecha de vencimiento', 'PE Tipo Comp Pago',
        'PE No Serie Comp Pago', 'PE No Comp Pago',
        'PE - Tipo Doc Identidad Cliente', 'Número de identificación nacional',
        'Nombre de la empresa', 'Importe (moneda extranjera)',
        'Custom Tax Total', 'Total Transaccion', 'Moneda: Símbolo de moneda',
        'Tipo de cambio'
    ]]

)

In [29]:
# Construir PEReporteVentas final

reporte_final = (
    pd.concat([a, a_], axis=0)
    .assign(**{
        'Número de identificación nacional': lambda x: x['Número de identificación nacional'].astype(str)
    })
    .merge(b.loc[:,['Foliador','description','status','concepto','codigo','porcentaje']], on=['Foliador'], how='left')
    .sort_values(['PE No Comp Pago'])
    .loc[:, [
        'Período contable: Fecha de inicio', 'Número de documento', 'Foliador',
        'Fecha', 'Fecha de vencimiento', 'PE Tipo Comp Pago',
        'PE No Serie Comp Pago', 'PE No Comp Pago',
        'PE - Tipo Doc Identidad Cliente', 'Número de identificación nacional',
        'Nombre de la empresa', 'Importe (moneda extranjera)',
        'Custom Tax Total', 'Total Transaccion', 'Moneda: Símbolo de moneda',
        'Tipo de cambio', 'description', 'status', 
        'PE Tipo Comp Pago NOTA', 'PE No Serie Comp Pago NOTA', 'PE No Comp Pago NOTA', 
        'Fecha Factura', 'Estado base', '# ORDEN VENTA', 'Creado desde', 
        'ID de transacción', 'Orden de Venta - Referencia Bob',
        'concepto', 'codigo', 'porcentaje'
    ]]
    .assign(**{
        'Tipo': 
            lambda x: np.select([
                            x['Número de documento'].str.contains('PE-SA|PE-SC')
                        ], [
                            'Autodetracción'
                        ], 
                            default=''
                        ),
        'Comision': 
            lambda x: np.select([
                            x['description'].astype(str).str.lower().str.contains('nac|nat')
                        ], [
                            'Si'
                        ], 
                            default='No'
                        ),
        'Seller_Tipo': 
            lambda x: np.select([
                            x['Número de identificación nacional'].isin(['20508565934','20100128056','20112273922'])
                        ], [
                            '1P'
                        ], 
                            default='3P'
                        )                                
    })
)

In [31]:
# Generar hoja data

data = (
    reporte_final
    .query("Tipo=='Autodetracción'")
    .query("Comision=='Si'")
    .query("Seller_Tipo=='3P'")
    .drop([
       'PE Tipo Comp Pago NOTA',
       'PE No Serie Comp Pago NOTA', 'PE No Comp Pago NOTA', 'Fecha Factura',
       'Estado base', '# ORDEN VENTA', 'Creado desde', 'ID de transacción',
       'Orden de Venta - Referencia Bob', 'Tipo', 'Comision', 'Seller_Tipo'
    ], axis=1)
    .drop(['Total Transaccion'], axis=1)  # Reemplazar Total Transaccion
    .merge(b.loc[:, ['Foliador','Total Transaccion']].drop_duplicates(), on=['Foliador'], how='left')
    .loc[:,[
       'Período contable: Fecha de inicio', 'Número de documento', 'Foliador',
       'Fecha', 'Fecha de vencimiento', 'PE Tipo Comp Pago',
       'PE No Serie Comp Pago', 'PE No Comp Pago',
       'PE - Tipo Doc Identidad Cliente', 'Número de identificación nacional',
       'Nombre de la empresa', 'Importe (moneda extranjera)',
       'Custom Tax Total', 'Total Transaccion', 'Moneda: Símbolo de moneda',
       'Tipo de cambio', 'description', 'status', 
       'concepto', 'codigo', 'porcentaje'
    ]]
    .assign(**{
        'PERIODO': lambda x: pd.to_datetime(x['Período contable: Fecha de inicio'], infer_datetime_format=True, errors='coerce').dt.strftime('%Y%m'),
    })
)

In [73]:
(
    pivot_codigo
    .query("Foliador=='F001-492595'")
    .merge(max_porcentaje, on=['Foliador','porcentaje'], how='left')
    .query("max=='max'")
    .drop_duplicates(['Foliador'], keep='first')
    .drop(['max'], axis=1)
)

,PERIODO,Número de identificación nacional,Nombre de la empresa,Foliador,codigo,porcentaje,total
1,202401,10010936671,CALAMPA MOZOMBITE MERLY,F001-492595,037,0.12,173.79


In [58]:
max_porcentaje

,Foliador,max_porcentaje,max
0,F001-492595,0.12,max


In [ ]:
p

In [81]:
# Generar pivot final

## Monto por foliador

pivot = (
    data
    .groupby(['PERIODO','Número de identificación nacional','Nombre de la empresa','Foliador'])
    .agg(total=('Total Transaccion','max'))
    .reset_index()
)

## Codigo y porcentaje

pivot_codigo = (
    data
    .groupby(['PERIODO','Número de identificación nacional','Nombre de la empresa','Foliador','codigo','porcentaje'])
    .agg(total=('Total Transaccion','max'))
    .reset_index()
)

max_porcentaje = (
    pivot_codigo
    .groupby(['Foliador'], dropna=False)
    .agg(porcentaje=('porcentaje','max'))
    .assign(max='max')
    .reset_index()
)

pivot_codigo = (
    pivot_codigo
    .merge(max_porcentaje, on=['Foliador','porcentaje'], how='left')
    .query("max=='max'")
    .drop_duplicates(['Foliador'], keep='first')
    .drop(['max'], axis=1)
)

## RUC

pivot_ruc = (
    data
    .groupby(['PERIODO','Número de identificación nacional','Nombre de la empresa','Foliador'])
    .agg(total=('Total Transaccion','max'))
    .reset_index()
    .groupby(['PERIODO','Número de identificación nacional'])
    .agg(total_ruc=('total','sum'))
    .reset_index()
)

## Final

pivot_final = (
    pivot
    .merge(pivot_ruc, how='left', on=['PERIODO','Número de identificación nacional'])
    .query("total_ruc>=699")
    .drop(['total_ruc'], axis=1)
    .merge(pivot_codigo.loc[:, ['Foliador','codigo','porcentaje']], how='left', on=['Foliador'])
    .assign(**{
        'detraccion': lambda x: round(x['total']*x['porcentaje'], 0)
    })
)

In [83]:
# Vista proveedor clientes para txt

proveedor_clientes = (
    pivot_final
    .rename(columns={
        'Número de identificación nacional': 'NÚMERO',
        'Nombre de la empresa': 'NOMBRE / RAZON SOCIAL',
        'codigo': 'BIEN O SERVICIO',
        'detraccion': 'IMPORTE'
    })
    .assign(**{
        'TIPO': 6,
        'N° PROF.': '00',
        'TIPO OPERACIÓN': '01',
        'TIPO_C': '01',
        'SERIE': lambda x: x['Foliador'].str.split('-', n=1).str[0],
        'NÚMERO_C': lambda x: x['Foliador'].str.split('-', n=1).str[1]
    })
    .loc[:, [
        'TIPO', 'NÚMERO', 'NOMBRE / RAZON SOCIAL', 'N° PROF.',
        'BIEN O SERVICIO', 'IMPORTE', 'TIPO OPERACIÓN', 'PERIODO',
        'TIPO_C', 'SERIE', 'NÚMERO_C'
    ]]
)

importe = round(proveedor_clientes['IMPORTE'].sum(),0)
importe_txt = importe.astype(str).replace('.0','.00').replace('.','')

In [84]:
# Generar tabla para txt

## 1. Empresa

empresa_txt = (
    pd.DataFrame({
        'columna_1': ['P' + ruc + empresa],
        'n_lote': [n_lote],
        'importe_txt': [importe_txt]
    })
    .assign(**{
        'conector': 
            lambda x: np.select([
                            x['importe_txt'].str.len()==0,
                            x['importe_txt'].str.len()==1,
                            x['importe_txt'].str.len()==2,
                            x['importe_txt'].str.len()==3,
                            x['importe_txt'].str.len()==4,
                            x['importe_txt'].str.len()==5,
                            x['importe_txt'].str.len()==6,
                            x['importe_txt'].str.len()==7,
                            x['importe_txt'].str.len()==8,
                            x['importe_txt'].str.len()==9,
                            x['importe_txt'].str.len()==10,
                            x['importe_txt'].str.len()==11,
                            x['importe_txt'].str.len()==12,
                            x['importe_txt'].str.len()==13,
                            x['importe_txt'].str.len()==14,
                            x['importe_txt'].str.len()==15,
                            x['importe_txt'].str.len()==16
                        ], [
                            '0000000000000000',
                            '000000000000000',
                            '00000000000000',
                            '0000000000000',
                            '000000000000',
                            '00000000000',
                            '0000000000',
                            '000000000',
                            '00000000',
                            '0000000',
                            '000000',
                            '00000',
                            '0000',
                            '000',
                            '00',
                            '0',
                            ''
                        ], 
                        default='0000000000000000'
            )
    })
    .assign(**{
        'columna_2': lambda x: n_lote + x['conector'] + importe_txt
    })
    .loc[:, ['columna_1','columna_2']]
)

## 2. Clientes

clientes_txt = (
    proveedor_clientes
    .assign(**{
        'importe_txt': lambda x: x['IMPORTE'].astype(str).str.replace('\\.0','.00').str.replace('.',''),
        'numero_c_txt': lambda x: x['NÚMERO_C'].astype(str),
    })
    .assign(**{
        'conector_1': 
            lambda x: np.select([
                            x['importe_txt'].str.len()==0,
                            x['importe_txt'].str.len()==1,
                            x['importe_txt'].str.len()==2,
                            x['importe_txt'].str.len()==3,
                            x['importe_txt'].str.len()==4,
                            x['importe_txt'].str.len()==5,
                            x['importe_txt'].str.len()==6,
                            x['importe_txt'].str.len()==7,
                            x['importe_txt'].str.len()==8,
                            x['importe_txt'].str.len()==9,
                            x['importe_txt'].str.len()==10,
                            x['importe_txt'].str.len()==11,
                            x['importe_txt'].str.len()==12,
                            x['importe_txt'].str.len()==13,
                            x['importe_txt'].str.len()==14,
                            x['importe_txt'].str.len()==15,
                        ], [
                            '000000000000000',
                            '00000000000000',
                            '0000000000000',
                            '000000000000',
                            '00000000000',
                            '0000000000',
                            '000000000',
                            '00000000',
                            '0000000',
                            '000000',
                            '00000',
                            '0000',
                            '000',
                            '00',
                            '0',
                            ''
                        ], 
                        default='000000000000000'
            ),
        'conector_2': 
            lambda x: np.select([
                            x['numero_c_txt'].str.len()==0,
                            x['numero_c_txt'].str.len()==1,
                            x['numero_c_txt'].str.len()==2,
                            x['numero_c_txt'].str.len()==3,
                            x['numero_c_txt'].str.len()==4,
                            x['numero_c_txt'].str.len()==5,
                            x['numero_c_txt'].str.len()==6,
                            x['numero_c_txt'].str.len()==7,
                            x['numero_c_txt'].str.len()==8
                        ], [
                            '00000000',
                            '0000000',
                            '000000',
                            '00000',
                            '0000',
                            '000',
                            '00',
                            '0',
                            ''
                        ], 
                        default='00000000'
            )    
    })
    .assign(**{
        'columna_1': lambda x: x['TIPO'].astype(str) + x['NÚMERO'],
        'columna_2': lambda x: '0000000' + x['N° PROF.'] + x['BIEN O SERVICIO'] + nro_cuenta + 
                               x['conector_1'] + x['importe_txt'] + x['TIPO OPERACIÓN'] + 
                               x['PERIODO'] + x['TIPO_C'] + x['SERIE'] + 
                               x['conector_2'] + x['NÚMERO_C']
    })
    .loc[:, ['columna_1', 'columna_2']]
)

# 3. Txt final

final_txt = pd.concat([empresa_txt, clientes_txt], axis=0)

In [85]:
# Exportar txt

final_txt.to_csv(os.path.join(ruta_autodetracciones, 'B' + ruc + n_lote + '.txt'), sep='\t', index=False, header=False)